In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("hw05.ipynb")

<div class="alert alert-success" markdown="1">

#### Homework 5

# APIs and Regular Expressions

### EECS 398: Practical Data Science, Winter 2025

#### Due Tuesday, February 18th at 11:59PM
    
</div>

## Instructions

Welcome to Homework 5! In this homework, you will wrangle real-world data from APIs and work with regular expressions. We'll be making requests to APIs to build structured DataFrames from JSON, and write regular expressions to extract key information from messy data.

You are given 8 slip days throughout the semester to extend deadlines. See the [Syllabus](https://practicaldsc.org/syllabus) for more details. With the exception of using slip days, late work will not be accepted unless you have made special arrangements with your instructor.

To access this notebook, you'll need to clone our [public GitHub repository](https://github.com/practicaldsc/wn25/). The [Environment Setup](https://practicaldsc.org/env-setup) page on the course website walks you through the necessary steps. Once you're done, you'll submit your completed notebook to Gradescope.

Please start early and submit often. You can submit as many times as you'd like to Gradescope, and we'll grade your **most recent** submission. Remember that the public `grader.check` tests in your notebook are not comprehensive, and that your work will also be graded on hidden test cases on Gradescope after the submission deadline.

This homework is worth a total of **49 points**, 44 of which come from the autograder and 5 of which are for completing our Pre-Midterm Survey (Question 0). The number of points each question is worth is listed at the start of each question. **The four questions in the assignment are independent, so feel free to move around if you get stuck**. Tip: if you're using Jupyter Lab, you can see a Table of Contents for the notebook by going to View > Table of Contents.

There are no manually-graded questions in this homework, i.e. all of the questions are **[Autograded 💻]**. All you need to submit is this notebook.

<div class="alert alert-danger" markdown="1">
<tt>for</tt>-loops are <strong>allowed</strong> throughout this entire homework.

</div>

To get started, run the import cell above, plus the cell at the top of the notebook that imports and initializes `otter`.

In [ ]:
import os
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup

## Question 0: Pre-Midterm Survey 📬 <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">5 Points</div>

---

We'd like to get your feedback on how the class is going so far, now that we're just over a month in. We've put together a survey that asks you to provide feedback on all aspects of the course. You can provide as much or as little detail as you'd like. We expect it will take 15 minutes to complete.

This survey is **NOT anonymous**, since it's for class credit – it counts towards your score for Homework 5. The responses to the survey will be visible to course staff.

<center><h3>Access the survey <a href="https://docs.google.com/forms/d/e/1FAIpQLSdYwKD7Uyvlz2Mr8ExdAxRIpjdW46NsZ03sFMTh4_r0JPARkQ/viewform?usp=preview"><b>here</b></a>.</center>

We will manually add 5 points to your Homework 5 score after the deadline, once we verify that you submitted this survey. Make sure to sign in with your @umich.edu email.

We really appreciate your feedback, thanks! 😊

## Question 1: Stock Stats 🤑

---

You're aspiring for a finance job in Chicago, and decide to put your new data wrangling skills to the test by pulling stock data from an API and calculating various statistics. The API we will work in this question is hosted by Financial Modeling Prep, and can be found at https://site.financialmodelingprep.com/developer/docs#daily-chart-charts. Specifically, we will use the "**Daily Chart EOD**" endpoint – search for it at the linked page.

Some relevant definitions:
- Ticker: A short code that refers to a stock. For example, Apple's ticker is AAPL and Ford's ticker is F. 
- Open: The price of a stock at the beginning of a trading day.
- Close: The price of a stock at the end of a trading day.
- Volume: The total number of shares traded in a day.
- Percent change: The difference in price with respect to the original price, as a percentage.

To make requests to the aforementioned API, you will need an API key. **In order to get one, you will need to make an account at the website.** Once you've signed up, you can use the API key that comes with the free plan. It has a limit of 250 requests per day, which should be more than enough. You will have to encode your API key in the URL that you make requests to; see a complete example of such a request at the [**documentation**](https://site.financialmodelingprep.com/developer/docs#Stock-Historical-Price).

### Question 1.1 <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">4 Points</div>

Complete the implementation of the function `stock_history`, which takes in a string `ticker` and two integers, `year` and `month`, and returns a DataFrame containing the price history for that stock in that month. Keep all of the attributes that are returned by the API.

Example behavior is given below.

```python
>>> out = stock_history('F', 2024, 8)
>>> out.shape
(22, 13)

# August 31st was a Saturday, which is why it doesn't appear – US markets are closed on weekends!
>>> out.head()
```

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>date</th>
      <th>open</th>
      <th>high</th>
      <th>low</th>
      <th>close</th>
      <th>adjClose</th>
      <th>volume</th>
      <th>unadjustedVolume</th>
      <th>change</th>
      <th>changePercent</th>
      <th>vwap</th>
      <th>label</th>
      <th>changeOverTime</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>2024-08-30</td>
      <td>11.15</td>
      <td>11.23</td>
      <td>11.06</td>
      <td>11.19</td>
      <td>11.19</td>
      <td>44977100</td>
      <td>44977100</td>
      <td>0.04</td>
      <td>0.35874</td>
      <td>11.1575</td>
      <td>August 30, 24</td>
      <td>0.003587</td>
    </tr>
    <tr>
      <th>1</th>
      <td>2024-08-29</td>
      <td>11.02</td>
      <td>11.20</td>
      <td>10.99</td>
      <td>11.11</td>
      <td>11.11</td>
      <td>44989200</td>
      <td>44989200</td>
      <td>0.09</td>
      <td>0.81670</td>
      <td>11.0800</td>
      <td>August 29, 24</td>
      <td>0.008167</td>
    </tr>
    <tr>
      <th>2</th>
      <td>2024-08-28</td>
      <td>11.10</td>
      <td>11.19</td>
      <td>10.98</td>
      <td>11.04</td>
      <td>11.04</td>
      <td>35442200</td>
      <td>35442200</td>
      <td>-0.06</td>
      <td>-0.54054</td>
      <td>11.0775</td>
      <td>August 28, 24</td>
      <td>-0.005405</td>
    </tr>
    <tr>
      <th>3</th>
      <td>2024-08-27</td>
      <td>11.12</td>
      <td>11.22</td>
      <td>10.99</td>
      <td>11.14</td>
      <td>11.14</td>
      <td>44841000</td>
      <td>44841000</td>
      <td>0.02</td>
      <td>0.17986</td>
      <td>11.1175</td>
      <td>August 27, 24</td>
      <td>0.001799</td>
    </tr>
    <tr>
      <th>4</th>
      <td>2024-08-26</td>
      <td>11.32</td>
      <td>11.37</td>
      <td>11.07</td>
      <td>11.11</td>
      <td>11.11</td>
      <td>53070331</td>
      <td>53070331</td>
      <td>-0.21</td>
      <td>-1.86000</td>
      <td>11.2175</td>
      <td>August 26, 24</td>
      <td>-0.018600</td>
    </tr>
  </tbody>
</table>




Some guidance:
- Read the API [**documentation**](https://site.financialmodelingprep.com/developer/docs#Stock-Historical-Price) if you get stuck! In particular, [this page](https://site.financialmodelingprep.com/playground?url=daily-chart-charts) will help you craft the URL you need to make a request to, though you'll need to make your account first.
- To format the starting and ending dates you'll need to pass to the API, [`pd.date_range`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.date_range.html) might be helpful, along with `pd.to_datetime`.
- The [`requests.get`](https://docs.python-requests.org/en/master/user/quickstart/) function returns a Response object, not the data itself. Use the `json` method on the Response object to extract the relevant JSON, as we did in [Lecture 8](https://practicaldsc.org/resources/lectures/lec08/lec08-filled.html#APIs-and-JSON) (you don't need to `import json` to do this).
- You can instantiate a DataFrame using a sequence of dictionaries as input to `pd.DataFrame`. Once you've gotten your response object back, you're done 99% of the work involved with this question.

In [ ]:
def stock_history(ticker, year, month):
    ...
    
# Feel free to change this input to make sure your function works correctly.
stock_history('F', 2024, 8).head()

In [ ]:
grader.check("q01_01")

### Question 1.2 <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">3 Points</div>

Create a function `stock_stats` that takes in a DataFrame outputted by `stock_history` and returns a **tuple** of two numbers:
1. The percent change of the stock throughout the month as a **percentage**.
2. An estimate of the total transaction volume **in billion of dollars** for that month.

Example behavior is given below.

```python
>>> stock_stats(stock_history('F', 2024, 8))
('+3.04%', '13.16B')
```

Both values in the tuple should be **strings** that contain numbers rounded to two decimal places. Add a plus or minus sign in front of the percent change, and make sure that the total transaction volume string ends in a `'B'`. Both strings should **always** have two decimal places, e.g. if a percentage change is 3% exactly, render it as `'+3.00%'`. Python [string formatting](https://docs.python.org/3/tutorial/inputoutput.html) will be helpful.

Let's illustrate both calculations. For example, suppose there are only three days in March – March 1st, March 2nd, and March 3rd. If BYND (Beyond Meat) opens at \\$4 on March 1st and closes at \\$5 on March 3rd, its **percent change** for the month of March is: $$\frac{\$5-\$4}{\$4} = +25.00\%$$

That is, when computing the percent change, use the opening price on the first day of the month as the starting price and the closing price on the last day of the month as the ending price.

<br>

**To compute the total transaction volume**, assume that on any given day, the average price of a share is the midpoint of the high and low price for that day, i.e. $\frac{\text{high} + \text{low}}{2}$.

$$ \text{Estimated Total Transaction Volume (in dollars)} = \text{Volume (number of shares traded)} \cdot \text{Average Price} $$




Suppose the high and low prices and volumes of BYND on each day are given below.
- March 1st: high \\$5, low \\$3, volume 500 million (0.5 billion).
- March 2nd: high \\$5.5, low \\$2.5, volume 1 billion.
- March 3rd: high \\$5.25, low \\$4, volume 500 million (0.5 billion).

Then, the estimated total transaction volume is:
$$\left( \frac{\$5 + \$3}{2} \cdot 0.5 B \right) + \left( \frac{\$5.5 + \$2.5}{2} \cdot 1 B \right) + \left( \frac{\$5.25 + \$4}{2} \cdot 0.5 B \right) = 8.3125B$$

In [ ]:
def stock_stats(history):
    ...
    
# Feel free to change this input to make sure your function works correctly.
stock_stats(stock_history('F', 2024, 8))

In [ ]:
grader.check("q01_02")

## Question 2: Comment Threads 🧵 <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">6 Points</div>

---

You regularly browse [Hacker News](https://news.ycombinator.com/) to keep up with the latest news in tech. An example link to a Hacker News article is https://news.ycombinator.com/item?id=18344932. Note that this article has 18 comments and has a `storyid` of 18344932. 

The problem now is that you don't have internet access on your phone during your morning commute to school, so you want to save the interesting stories' comment threads beforehand locally. You find their [API documentation](https://github.com/HackerNews/API) and get to work.

Complete the implementation of the function `get_comments`, which takes in a `storyid` and returns a DataFrame of all the comments below the news story.

Example behavior is given below.

```python
>>> get_comments(18344932).head()
```

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>id</th>
      <th>by</th>
      <th>parent</th>
      <th>text</th>
      <th>time</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>18380397</td>
      <td>valyala</td>
      <td>18344932</td>
      <td>TimescaleDB is great for storing time series c...</td>
      <td>2018-11-05 06:53:19</td>
    </tr>
    <tr>
      <th>1</th>
      <td>18346406</td>
      <td>msiggy</td>
      <td>18344932</td>
      <td>I&amp;#x27;m excited to give this database a try i...</td>
      <td>2018-10-31 15:20:22</td>
    </tr>
    <tr>
      <th>2</th>
      <td>18348601</td>
      <td>sman393</td>
      <td>18344932</td>
      <td>Can this be used side by side on normal Postgr...</td>
      <td>2018-10-31 19:29:39</td>
    </tr>
    <tr>
      <th>3</th>
      <td>18348631</td>
      <td>RobAtticus</td>
      <td>18348601</td>
      <td>Yep, absolutely. Regular PostgreSQL tables coe...</td>
      <td>2018-10-31 19:34:52</td>
    </tr>
    <tr>
      <th>4</th>
      <td>18348984</td>
      <td>sman393</td>
      <td>18348631</td>
      <td>Good to hear! how does the current TimescaleDB...</td>
      <td>2018-10-31 20:23:46</td>
    </tr>
  </tbody>
</table>

As you see above, the DataFrame that `get_comments` returns should have 5 columns:
- `'id'`: The unique ID of the comment.
- `'by'`: The author of the comment.
- `'parent'`: The unique ID of the comment this comment is replying to.
- `'text'`: The actual comment.
- `'time'`: When the comment was created (in `pd.Timestamp` format).

Some guidance:
- The URL to make requests to is `'https://hacker-news.firebaseio.com/v0/item/{id}.json'`, however, the `{id}` should be replaced with the ID of the article or page you are trying to access. 
- Again, do not `import json` – instead, use the `json` method on the Response object you get back.
- **Make sure the comments in your DataFrame appear in the same order that they do on the page, from top to bottom.** Use depth-first search when traversing the comments tree. You will have to do this manually, since you cannot use BeautifulSoup (which is only for HTML documents, not JSON objects). Haven't taken (or finished) EECS 281 and don't know what depth-first search is? Don't worry – the video linked in the green box below will walk through the general idea.
- Make sure the length of your returned DataFrame is the same as value for the `'descendants'` key in the response JSON, both of which correspond to the number of comments for the story.
- You should ignore "dead" comments (you will know them when you see them), as well as "dead" comments' children. 
- Remember, you're allowed to use loops in this function (both `for`-loops and, **hint**, other types of loops!). You may also want to create at least one helper function.

<div class="alert alert-block alert-success">
You may find <a href="https://www.youtube.com/watch?v=uOfwW-onmpc"><b>this hint video 🎥</b></a> helpful!

Also, the following Python behavior may be useful:

```python
>>> places = [280, 203, 398, 492]
>>> places.pop(0)
280
>>> places
[203, 398, 492]

>>> places = [183, 101] + places
>>> places
[183, 101, 203, 398, 492]

```
 
</div>

Some extra motivation from Max, a student from last semester:

> "I also wanted to share a recent interview experience. I had an online technical interview for a company, and they told me beforehand that the questions would be “general technical questions.” I wasn’t sure what to prepare for. It turned out the question was remarkably similar to Homework 5 Question 2, where we had to web scrape and use DFS/BFS to print out comments. In my interview, I had to web scrape and then traverse all the links on each page to print them out, my mind instantly went to Beautiful Soup and that question and the explanation video for BFS/DFS that we were given."

In [ ]:
def get_comments(storyid):
    ...
    
# Feel free to change this input to make sure your function works correctly.
# For example, test your code on newer Hacker News articles, like
# https://news.ycombinator.com/item?id=41660308!
get_comments(18344932)

In [ ]:
grader.check("q02")

## Question 3: Practice with Regular Expressions 📕

---

Regular expressions can be tricky, and the best way to gain familiarity with them is through lots of practice.

In this question, you will work through 10 parts, **each of which requires you to write a regular expression that matches strings that satisfy certain criteria**. You will do this by – as usual – completing the implementation of a function. In Questions 3.1 through 3.9, your function will take in a string and return `True` if the string follows the pattern and `False` otherwise.

- Make sure to take a close look at the examples for each function, as they provide useful guidance for the types of strings you should and shouldn't match.
- Make sure to refer to the [Regular Expression Resources](https://practicaldsc.org/resources/#regular-expressions) on the course website. In particular, we recommend having [regex101.com](https://regex101.com/) open while working, along with the [cheat sheet](https://practicaldsc.org/resources/other/berkeley-regex-reference.pdf).
- The number of points each part is worth tells you its relative difficulty level – some are worth <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">2 Points</div> and some are worth <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">3 Points</div>. If you're spending lots of time on exercises worth <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">2 Points</div>, take a close look at the syntax from [Lecture 9](https://practicaldsc.org/resources/lectures/lec09/lec09-filled.html), as there is probably an easier way of writing the necessary pattern!
- The 10 parts are all independent, and are **not** sorted by difficulty – some of the easiest parts are in the middle or towards the end!

### Question 3.1  <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">2 Points</div>

Write a regular expression that matches strings that have `'['` as the third character and `']'` as the sixth character. Example behavior is given below.

```python
>>> match_1("abcde]")
False

>>> match_1("ab[cde")
False

>>> match_1("ab[cd]ef")
True
```

In [ ]:
def match_1(string):
    pattern = ...

    # Do not edit the following code.
    return re.findall(pattern, string) != []

In [ ]:
grader.check("q03_01")

### Question 3.2 <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">2 Points</div>

Write a regular expression that matches strings that are phone numbers that start with `'(734)'` and follow the format `'(xxx) xxx-xxxx'` (`'x'` represents a digit). Example behavior is given below.

```python
>>> match_2("(734) 456-7890")
True

>>> match_2("(123) 456-7890")
False

>>> match_2("(734) 456-7890b")
False
```

Note that there is a space between `'(xxx)'` and `'xxx-xxxx'`!

In [ ]:
def match_2(string):
    pattern = ...

    # Do not edit the following code.
    return re.findall(pattern, string) != []

In [ ]:
grader.check("q03_02")

### Question 3.3 <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">2 Points</div>

Write a regular expression that matches strings that:
- are between 6 and 10 characters long (inclusive),
- contain only alphanumeric characters, whitespace and `'?'`, and
- end with `'?'`.

Example behavior is given below.

```python
>>> match_3('qw?ertsd?')
True

>>> match_3("ab   c ?")
True

>>> match_3(" adf!qes ?")
False

>>> match_3('wwwWW .? ')
False
```

Note that `'?'` is a special character.

In [ ]:
def match_3(string):
    pattern = ...

    # Do not edit the following code.
    return re.findall(pattern, string) != []

In [ ]:
grader.check("q03_03")

### Question 3.4 <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">3 Points</div>

Write a regular expression that matches strings with exactly two `'$'`, one of which is at the start of the string, such that:
- the characters between the two `'$'` can be anything (including nothing) except the lowercase letters `'a'`, `'b'`, and `'c'`, (and `'$'`), and
- the characters after the second `'$'` can only be the **lowercase or uppercase** letters `'a'`/`'A'`, `'b'`/`'B'`, and `'c'`/`'C'`, with every `'a'`/`'A'` before every `'b'`/`'B'`, and every `'b'`/`'B'` before every `'c'`/`'C'`. There **must be** at least one `'a'` or `'A'`, at least one `'b'` or `'B'`, and at least one `'c'` or `'C'`.

Example behavior is given below.

```python
>>> match_4("$!@#$aABc")
True

>>> match_4('$qw!!  $aaBC')
True

>>> match_4('$a$aABc')
False

>>> match_4('$!@$')
False
```

In [ ]:
def match_4(string):
    pattern = ...

    # Do not edit the following code.
    return re.findall(pattern, string) != []

In [ ]:
grader.check("q03_04")

### Question 3.5 <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">2 Points</div>

Write a regular expression that matches strings that represent valid Python file names, including the extension. For simplicity, assume that file names only contain letters, numbers, and underscores. Example behavior is given below.

```python
>>> match_5("eecs398.py")
True

>>> match_5('eecs398_.py')
True

>>> match_5("here is a Python file eecs398.py")
False

>>> match_5("eecs398+.py")
False
```

In [ ]:
def match_5(string):
    pattern = ...

    # Do not edit the following code.
    return re.findall(pattern, string) != []

In [ ]:
grader.check("q03_05")

### Question 3.6 <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">2 Points</div>

Write a regular expression that matches strings that:
- are made up of only lowercase letters and exactly one underscore (`'_'`), and
- have at least one lowercase letter on both sides of the underscore.

Example behavior is given below.

```python
>>> match_6("aab_cbbbc")
True

>>> match_6("zebra_d")
True

>>> match_6("aab_Abbbc")
False

>>> match_6("zebra_")
False
```

In [ ]:
def match_6(string):
    pattern = ...

    # Do not edit the following code.
    return re.findall(pattern, string) != []

In [ ]:
grader.check("q03_06")

### Question 3.7 <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">2 Points</div>

Write a regular expression that matches strings that start with and end with an underscore (`'_'`). Example behavior is given below.

```python
>>> match_7("_abc_")
True

>>> match_7("_ZeBr@45Din000!!!\b_")
True

>>> match_7("abc")
False

>>> match_7("_ncde")
False

>>> match_7("_") # Need at least two underscores!
False
```

In [ ]:
def match_7(string):
    pattern = ...

    # Do not edit the following code.
    return re.findall(pattern, string) != []

In [ ]:
grader.check("q03_07")

### Question 3.8 <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">2 Points</div>

Apple serial numbers are strings of length 1 or more that are made up of any characters, other than
- the uppercase letter `'O'`, 
- the lowercase letter `'i`', and 
- the number `'1'`.

Write a regular expression that matches strings that are valid Apple serial numbers. Example behavior is given below.

```python
>>> match_8('ASDJKL9380JKAL')
True

>>> match_8("ASJDKLFK0ASDo!!!!!!! !!!!!!!!!")
True

>>> match_8('iPhone 10')
False

>>> match_8("hi ASDJKL9380JKAL")
False
```

In [ ]:
def match_8(string):
    pattern = ...

    # Do not edit the following code.
    return re.findall(pattern, string) != []

In [ ]:
grader.check("q03_08")

### Question 3.9 <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">3 Points</div>

Suppose DataID numbers are formatted as `'SC-NN-CCC-NNNN'`, where 
- SC represents state code in uppercase (e.g. `'MI'`),
- NN represents a number with 2 digits (e.g. `'98'`),
- CCC represents a three letter city code in uppercase (e.g. `'DTW'`), and
- NNNN represents a number with 4 digits (e.g. `'1998'`).

Write a regular expression that matches strings that are DataID numbers corresponding to the cities of `'DTW'` (Detroit) or `'LAN'` (Lansing), or the state of `'TX'` (Texas). Assume that there is only one city named `'DTW'` and only one city named `'LAN'`.

Example behavior is given below.

```python
>>> match_9('MI-32-LAN-1232')
True

>>> match_9('TX-32-DTW-1232')
True

# Lansing is not in California!
>>> match_9('CA-32-LAN-1232')
False

>>> match_9('mI-32-LAN-1232')
False
```

In [ ]:
def match_9(string):
    pattern = ...

    # Do not edit the following code.
    return re.findall(pattern, string) != []

In [ ]:
grader.check("q03_09")

### Question 3.10 <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">3 Points</div>

In this final part, your task involves more than writing a single regular expression.

Complete the implementation of the function `match_10`, which takes in a string (`string`) and:
- converts the string to lowercase,
- removes all non-alphanumeric characters (i.e. removes everything that is not in the `\w` character class), and the letter `'a'`, and
- returns a list of every **non-overlapping** three-character substring in the remaining string, starting from the beginning of the string.
   
Example behavior is given below.

```python
>>> match_10('Ab..DEF')
['bde']

>>> match_10('FINALS are COMING A')
['fin', 'lsr', 'eco', 'min']

>>> match_10('h9i9hOWW44areY@')
['h9i', '9ho', 'ww4', '4re']
```

Here's how `match_10` should process `'Ab..DEF'`:

1. Convert to lowercase: `'ab..def'`.
2. Remove non-alphanumeric characters and the letter `'a'`: `'bdef'`.
3. Starting from the beginning of the string, there is only a single non-overlapping three character substring: `'bde'`. Hence, we return `['bde']`.

Some guidance: 
- Perform your operations in the exact order described above, otherwise your code may not pass all the tests.
- Don't use a `for`-loop. You'll need to use `re.sub` in addition to `re.findall`.

In [ ]:
def match_10(string):
    ...

In [ ]:
grader.check("q03_10")

## Question 4: Capture Groups 📡

---

The dataset stored in `data/messy.txt` contains personal information from a fictional website that a user scraped from web server logs. Within this dataset, there are four fields that are of interest to you:
1. Social Security Numbers
1. Bitcoin Addresses
1. Email Addresses 
1. Street Addresses

Your job is to use `re.findall` to extract out the relevant pieces of information from `messy.txt`. **Since this data is very messy, your function will be allowed to miss ~5% of the records in each list. Good spot checking using certain useful substrings (e.g. `'@'` for emails) should help assure correctness!** As usual, your functions will be tested on a sample of the file `messy.txt`.

Note that there are multiple "delimiters" (separators) in use in the file; there are few enough of them that you can safely determine what they are. **Before attempting any of the parts here, open `data/messy.txt` in your favorite text editor and explore!**

### Question 4.1 <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">2 Points</div>

Complete the implementation of the function `extract_ssns`, which takes in a string (`string`) containing the contents of a server log file and returns the Social Security Numbers in the file as a list. Example behavior is given below.

```python
>>> extract_ssns('bitcoin:1BvBMSEYstWetqTFn5Au4m4GFg7xJaNVN2$jk%^3\t,test@test55.umich.edu,lkj5r%ji|ssn:423-01-9575,530 High Street')
['423-01-9575']

>>> out = extract_ssns(open('data/messy.txt', encoding='utf8').read())
>>> out[0]
'380-09-9403'
```

Some guidance:
- For our purposes, an SSN is a string of the form 3 digits-2 digits-4 digits.
- The returned list should not contain any empty strings or the string `'null'`.

In [ ]:
def extract_ssns(string):
    ...
    
# To test your work, first run:
extract_ssns('bitcoin:1BvBMSEYstWetqTFn5Au4m4GFg7xJaNVN2$jk%^3\t,test@test55.umich.edu,lkj5r%ji|ssn:423-01-9575,530 High Street')
# Then, once that works, uncomment:
# extract_ssns(open('data/messy.txt', encoding='utf8').read())

In [ ]:
grader.check("q04_01")

### Question 4.2  <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">2 Points</div>

Complete the implementation of the function `extract_bitcoin_addresses`, which takes in a string (`string`) containing the contents of a server log file and returns the Bitcoin addresses in the file as a list. Example behavior is given below. 

```python
>>> extract_bitcoin_addresses('bitcoin:1BvBMSEYstWetqTFn5Au4m4GFg7xJaNVN2$jk%^3\t,test@test55.umich.edu,lkj5r%ji|ssn:423-01-9575,530 High Street')
['1BvBMSEYstWetqTFn5Au4m4GFg7xJaNVN2']

>>> out = extract_bitcoin_addresses(open('data/messy.txt', encoding='utf8').read())
>>> out[0]
'18A8rBU3wvbLTSxMjqrPNc9mvonpA4XMiv'
```

Some guidance:
- Assume Bitcoin addresses are alphanumeric strings.
- The returned list should not contain any empty strings or the string `'null'`.

In [ ]:
def extract_bitcoin_addresses(string):
    ...
    
# To test your work, first run:
extract_bitcoin_addresses('bitcoin:1BvBMSEYstWetqTFn5Au4m4GFg7xJaNVN2$jk%^3\t,test@test55.umich.edu,lkj5r%ji|ssn:423-01-9575,530 High Street')
# Then, once that works, uncomment:
# extract_bitcoin_addresses(open('data/messy.txt', encoding='utf8').read())

In [ ]:
grader.check("q04_02")

### Question 4.3 <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">2 Points</div>

Complete the implementation of the function `extract_emails`, which takes in a string (`string`) containing the contents of a server log file and returns the email addresses in the file as a list. Example behavior is given below.

```python
>>> extract_emails('bitcoin:1BvBMSEYstWetqTFn5Au4m4GFg7xJaNVN2$jk%^3\t,test@test55.umich.edu,lkj5r%ji|ssn:423-01-9575,530 High Street')
['test@test55.umich.edu']

>>> out = extract_emails(open('data/messy.txt', encoding='utf8').read())
>>> out[0]
'dottewell0@gnu.org'
```

Some guidance:
- Assume that the usernames and domain names in an email address are alphanumeric. Domain names don't need to end in `'.com'` – assume that all parts of a domain name, including the very end, can be made up of any alphanumeric characters.
- The returned list should not contain any empty strings or the string `'null'`. (It likely won't by default, but we've included this instruction in all four parts of this question.)

In [ ]:
def extract_emails(string):
    ...
    
# To test your work, first run:
extract_emails('bitcoin:1BvBMSEYstWetqTFn5Au4m4GFg7xJaNVN2$jk%^3\t,test@test55.umich.edu,lkj5r%ji|ssn:423-01-9575,530 High Street')
# Then, once that works, uncomment:
# extract_emails(open('data/messy.txt', encoding='utf8').read())

In [ ]:
grader.check("q04_03")

### Question 4.4 <div style="display:inline-block; vertical-align: middle; padding:7px 7px; font-size:10px; font-weight:light; color:white; background-color:#e84c4a; border-radius:7px; text-align:left;">2 Points</div>

Complete the implementation of the function `extract_street_addresses`, which takes in a string (`string`) containing the contents of a server log file and returns the street addresses in the file as a list. Example behavior is given below.

```python
>>> extract_street_addresses('bitcoin:1BvBMSEYstWetqTFn5Au4m4GFg7xJaNVN2$jk%^3\t,test@test55.umich.edu,lkj5r%ji|ssn:423-01-9575,530 High Street')
['530 High Street']

>>> out = extract_street_addresses(open('data/messy.txt', encoding='utf8').read())
>>> out[0]
'814 Monterey Court'
```

As before, the returned list should not contain any empty strings or the string `'null'`.

In [ ]:
def extract_street_addresses(string):
    ...

# To test your work, first run:
extract_street_addresses('bitcoin:1BvBMSEYstWetqTFn5Au4m4GFg7xJaNVN2$jk%^3\t,test@test55.umich.edu,lkj5r%ji|ssn:423-01-9575,530 High Street')
# Then, once that works, uncomment:
# extract_street_addresses(open('data/messy.txt', encoding='utf8').read())

In [ ]:
grader.check("q04_04")

## Finish Line 🏁

Congratulations! You're ready to submit Homework 5. Remember that all you need to submit is this notebook; there is no separate PDF to submit, since there were no written questions.

To submit your homework:

1. Select `Kernel -> Restart & Run All` to ensure that you have executed all cells, including the test cells.
2. Read through the notebook to make sure everything is fine and all tests passed.
3. Download your notebook using `File -> Download as -> Notebook (.ipynb)`, then upload your notebook to Gradescope under "Homework 5".
4. Stick around while the Gradescope autograder grades your work. Make sure you see that all **public tests** have passed on Gradescope. **Remember that homeworks have hidden tests, which you will not see your scores on until a few days after the deadline!**
5. Check that you have a confirmation email from Gradescope and save it as proof of your submission.